In [ ]:
import pandas as pd
import altair as alt
import requests as req
import numpy as np
from scipy import stats
from collections import defaultdict
import math
alt.data_transformers.enable("vegafusion")

In [ ]:
openml_url = "https://www.openml.org/api/v1/json/data/43633"
dataset = req.get(openml_url).json()['data_set_description']

### Pre-Processing

In [ ]:
df = pd.read_parquet(dataset['parquet_url'])
df

In [ ]:
df = df.dropna()

sale_date_list = df['SALE_DATE'].str.split(" ").to_list()
sale_year_list = []
sale_month_list = []
month_name = []
month_dict ={
    1: 'Jan',
    2: 'Feb',
    3: 'Mar',
    4: 'Apr',
    5: 'May',
    6: 'Jun',
    7: 'Jul',
    8: 'Aug',
    9: 'Sep',
    10: 'Oct',
    11: 'Nov',
    12: 'Dec'
}

for date in sale_date_list:
    sale_year_list.append(int(date[0].split("-")[0]))
    sale_month_list.append(int(date[0].split("-")[1]))
    month_name.append(month_dict[int(date[0].split("-")[1])])

df['SALE_MONTH'] = sale_month_list
df['SALE_YEAR'] = sale_year_list
df['MONTH_NAME']  = month_name
borough_dict = {
    1: 'Manhattan',
    2: 'The Bronx',
    3: 'Brooklyn',
    4: 'Queens',
    5: 'Staten Island'
}

df['BOROUGH_NAME'] = df['BOROUGH'].map(borough_dict)
df

In [ ]:
df = df[np.abs(stats.zscore(df['SALE_PRICE'])) < 3]
df = df.query('SALE_PRICE > 0')
df

In [ ]:
sale_dict = defaultdict(lambda: 0)
sold_addresses = df['ADDRESS'].to_list()

for address in sold_addresses:
    sale_dict[address] += 1

df['SALES'] = df['ADDRESS'].map(sale_dict)

mean_price_per_address = dict(df.groupby('ADDRESS')['SALE_PRICE'].mean())
df['MEAN_ADDRESS_PRICE'] = df['ADDRESS'].map(mean_price_per_address)

def log_calc(x):
    if x == 0: return 0
    else: return math.log(x)

df['SALE_PRICE_log'] = df['SALE_PRICE'].map(log_calc)
df['LAND_SQUARE_FEET_log'] = df['LAND_SQUARE_FEET'].map(log_calc)
df['GROSS_SQUARE_FEET_log'] = df['GROSS_SQUARE_FEET'].map(log_calc)
df['MEAN_ADDRESS_PRICE_log'] = df['MEAN_ADDRESS_PRICE'].map(log_calc)

df

In [ ]:
def rename(x):
    x = x.rstrip()

    if len(x) == 0: return 'UNKNOWN'

    return x.replace("  ", " ")

df['BUILDING_CLASS_CATEGORY'] = df['BUILDING_CLASS_CATEGORY'].map(rename)

### First Iteration Plots

In [ ]:
upper = alt.Chart(df).mark_boxplot(size=50).encode(
    x='SALE_YEAR:N',
    y='SALE_PRICE:Q' 
).properties(width=1000)

lower = alt.Chart(df).mark_boxplot(clip=True, size=50).encode(
    y = alt.Y('SALE_PRICE:Q', scale=alt.Scale(domain=[0, 2000000])),
    x='SALE_YEAR:N' 
).properties(width=1000)

alt.vconcat(upper, lower)

In [ ]:
right = alt.Chart(df).mark_bar(clip=True, size=20).encode(
    y = alt.Y('count()', scale=alt.Scale(domain=[70000, 110000])),
    x= alt.X('MONTH_NAME:O', sort=alt.EncodingSortField(field="SALE_MONTH", order='ascending'))
).properties(width=500, height=500)

left = alt.Chart(df).mark_bar(clip=True, size=20).encode(
    y = alt.Y('median(SALE_PRICE)', scale=alt.Scale(domain=[450000, 500000])),
    x= alt.X('MONTH_NAME:O', sort=alt.EncodingSortField(field="SALE_MONTH", order='ascending'))
).properties(width=500, height=500)

alt.hconcat(right, left, spacing=30)

In [ ]:
alt.Chart(df).mark_line(clip=True).encode(
    y = alt.Y('median(SALE_PRICE)', scale=alt.Scale(domain=[200000, 2000000])),
    x = 'SALE_YEAR:O',
    color = 'BOROUGH_NAME'
).properties(width=500, height=500)

## Final Iteration

In [ ]:
slider = alt.binding_range(min=df['SALE_YEAR'].min(), max=df['SALE_YEAR'].max(), step=1, name='Year ')
selector = alt.param(name='SelectorName', value=2003, bind=slider)
selection2 = alt.selection_interval(bind='scales')

alt.Chart(df).add_params(
    selector,
    selection2
).transform_aggregate(
    yearly_median_price = 'median(SALE_PRICE)',
    yearly_mean_land = 'mean(LAND_SQUARE_FEET)',
    groupby=['NEIGHBORHOOD', 'SALE_YEAR', 'BOROUGH_NAME']
).mark_point(clip=True).encode(
    y = alt.Y('yearly_median_price:Q', scale=alt.Scale(domain=[0, 1000000])),
    x = alt.X('yearly_mean_land:Q', scale=alt.Scale(domain=[0, 10000])),
    color = 'BOROUGH_NAME:N',
    detail = 'NEIGHBORHOOD:N',
    tooltip='NEIGHBORHOOD:N'
).transform_filter(
    (alt.datum.SALE_YEAR == selector)
).properties(width=1000, height=500)

In [ ]:
min_sale_num = 6
aux_df = df.query(f'SALES >= {min_sale_num}')

slider = alt.binding_range(min=0, max=9, step=1, name='Sales 1s')
selector = alt.param(name='SelectorName_1s', value=min_sale_num, bind=slider)

slider_tens = alt.binding_range(min=0, max=90, step=10, name='Sales 10s')
selector_tens = alt.param(name='SelectorName_10s', value=0, bind=slider_tens)

slider_hundreds = alt.binding_range(min=0, max=900, step=100, name='Sales 100s')
selector_hundreds = alt.param(name='SelectorName_100s', value=0, bind=slider_hundreds)

top = alt.Chart(aux_df).mark_rect().encode(
    y = alt.Y('BOROUGH_NAME:N'),
    x = alt.X('BUILDING_CLASS_CATEGORY:N'),
    color=alt.Color('count():Q', legend=alt.Legend(orient='top', direction='horizontal', title='Ocorrências em Logaritimo')).scale(type="log"),
    tooltip='count():Q'
).transform_filter(
    (alt.datum.SALES >= (selector + selector_tens + selector_hundreds))
).properties(height = 500, width = 1480)


bottom = alt.Chart(aux_df).add_params(
    selector, selector_tens, selector_hundreds
).transform_aggregate(
    groupby=['ADDRESS', 'MEAN_ADDRESS_PRICE', 'SALES']
).mark_boxplot().encode(
    x = alt.X('MEAN_ADDRESS_PRICE:Q', scale=alt.Scale(domain=[0, aux_df['MEAN_ADDRESS_PRICE'].max()])),
    tooltip='ADDRESS:N'
).transform_filter(
    (alt.datum.SALES >= (selector + selector_tens + selector_hundreds))
).properties(width=1480)

alt.vconcat(top, bottom, spacing=30) 